In [1]:
import numpy as np
import tensorflow as tf

In [2]:
test_output = np.random.rand(4, 32)
test_output

array([[0.48662477, 0.85811425, 0.58293108, 0.97059916, 0.00774773,
        0.80666061, 0.91114613, 0.05836124, 0.4875604 , 0.9757836 ,
        0.21615148, 0.22762642, 0.89543612, 0.07418308, 0.93768228,
        0.87579256, 0.24849466, 0.26565475, 0.40037376, 0.27835487,
        0.70131639, 0.64707274, 0.37467695, 0.40675179, 0.38007618,
        0.59974902, 0.2412411 , 0.78327804, 0.89182006, 0.74456722,
        0.36168603, 0.79617879],
       [0.91202145, 0.90017815, 0.61835591, 0.48806282, 0.11917384,
        0.96495233, 0.91916896, 0.08978088, 0.8888787 , 0.70807142,
        0.49013597, 0.15039863, 0.93828634, 0.61837416, 0.88455815,
        0.00749638, 0.64235936, 0.85831509, 0.56418271, 0.22562489,
        0.99523993, 0.61231703, 0.61185212, 0.43401731, 0.76996104,
        0.25719184, 0.68979226, 0.15054809, 0.19941846, 0.56089769,
        0.80079559, 0.3408441 ],
       [0.10986075, 0.48554537, 0.50574521, 0.85815962, 0.62421124,
        0.82138793, 0.52957885, 0.22630943, 0.8568

In [65]:
test_labels = np.random.randint(0,2, (4,))
test_labels = tf.cast(test_labels, 'double')
test_labels

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([1., 0., 0., 1.])>

In [160]:
class combo_method():
    
    def __init__(self):
        pass
    
    def forward(self, batch_features, batch_labels):
        self.batch_features = batch_features
        self.batch_labels = batch_labels
        
        self.distances = self.calc_distance_mtx(batch_features, batch_features)

        class_0 = tf.cast(batch_labels[:] == 0, 'double')
        class_1 = tf.cast(batch_labels[:] == 1, 'double')

        aggregate = tf.stack([tf.reduce_sum(tf.multiply(self.distances, class_0), 1), tf.reduce_sum(tf.multiply(self.distances, class_1), 1)], axis=1)
        self.softmax = tf.truediv(aggregate, tf.reduce_sum(aggregate, 1, keepdims=True))
        
        return self.softmax
    
    def backward(self, loss):
        self.loss = loss
        dL_dS = -tf.math.divide_no_nan(self.batch_labels, self.softmax[:,0]) + tf.math.divide_no_nan((1-self.batch_labels),(1-self.softmax[:,0]))
        
        dS_dZ = tf.tensordot(-self.softmax[:,0],self.softmax[:,0], axes=0)
        dS_dZ = tf.linalg.set_diag(dS_dZ, tf.math.multiply_no_nan(self.softmax[:,0],(1-self.softmax[:,0])))

        dD_dX = np.ones(self.batch_features.shape)
        for h in range(self.batch_features.shape[1]):
            dz_dx = np.ones(self.distances.shape) 
            for i in range(self.distances.shape[0]):
                for j in range(self.distances.shape[1]):
                    dz_dx[i,j] = 2*(self.batch_features[i,h] - self.batch_features[j,h])
            dD_dX[:,h] = dz_dx.sum(axis=1)
 
        return dD_dX
    
    def calc_distance_mtx(self, A, B):
        """
        Computes squared pairwise distances between each elements of A and each elements of B.
        Args:
        A,    [m,d] matrix
        B,    [n,d] matrix
        Returns:
        D,    [m,n] matrix of pairwise distances
        """
        with tf.compat.v1.variable_scope('pairwise_dist'):
            # squared norms of each row in A and B
            na = tf.reduce_sum(tf.square(A), 1)
            nb = tf.reduce_sum(tf.square(B), 1)

            # na as a row and nb as a co"lumn vectors
            na = tf.reshape(na, [-1, 1])
            nb = tf.reshape(nb, [1, -1])

            # return pairwise euclidead difference matrix
            D = tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0.0)
        return D

In [161]:
combo = combo_method()
pred_labels = combo.forward(test_output, test_labels)
print(pred_labels[:,0])

tf.Tensor([0.65227936 0.3319245  0.36381357 0.67616299], shape=(4,), dtype=float64)


In [162]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
TL = loss(test_labels, pred_labels[:,0])
TL

<tf.Tensor: shape=(), dtype=float64, numpy=0.6486899256706238>

In [163]:
combo.backward(TL)

array([[ 0.82178488,  0.97905442, -0.29173334,  2.6316093 , -2.97299287,
         1.00738066,  0.74604467, -0.39117489, -1.26513516,  1.59396037,
        -1.94961612, -1.92582807,  1.42479895, -3.60706235,  3.09205171,
         3.20823016, -0.2759109 , -1.73093684, -0.08973434, -0.34704177,
        -1.05566733, -0.80731667, -1.10400543,  0.10405632, -1.64739199,
         0.93250303, -1.715697  ,  2.98784381,  1.63429643, -0.19203757,
        -2.00393065,  2.76355236],
       [ 4.22495831,  1.31556556, -0.00833473, -1.22868144, -2.08158396,
         2.27371442,  0.81022735, -0.13981782,  1.94541126, -0.54773706,
         0.24225978, -2.54365037,  1.7676007 ,  0.74646629,  2.66705864,
        -3.73813926,  2.87500667,  3.01034591,  1.22073725, -0.7688816 ,
         1.29572101, -1.08536234,  0.79339589,  0.3221805 ,  1.47168691,
        -1.8079544 ,  1.87271228, -2.07399579, -3.9049163 , -1.66139384,
         1.50894582, -0.87912511],
       [-2.19232726, -2.00149665, -0.90922035,  1.7320

In [152]:
class arun_knn():
    
    def __init__(self):
        pass
    
    def forward(self, batch_features, batch_labels):
        batch_features = np.array(batch_features)
        #print(batch_features.shape)
        self.batch_features = batch_features
        self.batch_labels = batch_labels
        
        self.distance = self.pairwise_dist(self.batch_features,self.batch_features)
        inverse_distance = np.ones(self.distance.shape)
        inverse_distance = np.divide(1,self.distance, out=inverse_distance, where=self.distance!=0)
        #print(inverse_distance)
        Z = []
        for i in range(len(inverse_distance)):
            z0 = inverse_distance[i,self.batch_labels == 0].sum()
            z1 = inverse_distance[i,self.batch_labels == 1].sum()
            Z.append([z0,z1])
        self.Z = tf.convert_to_tensor(Z)
        self.S = tf.keras.activations.softmax(self.Z, axis=1)
        return self.S
    
    def backward(self, loss):
        self.loss = loss
        dL_dS = -tf.math.divide_no_nan(self.batch_labels, self.S[:,1]) + tf.math.divide_no_nan((1-self.batch_labels),(1-self.S[:,1]))
        #dL_dS = tf.gradients(loss, self.S)
        #print("Loss Gradients:")
        #print(dL_dS)
        
        ds1_dz0 = tf.math.multiply_no_nan(-self.S[:,1],self.S[:,0])
        ds1_dz1 = tf.math.multiply_no_nan(self.S[:,1],(1-self.S[:,1]))
        
        distance_0 = np.copy(self.distance)
        distance_1 = np.copy(self.distance)
        distance_0[:,self.batch_labels==1] =  0
        distance_1[:,self.batch_labels==0] =  0
        
        dz0_dd = np.ones(distance_0.shape)
        dz0_dd = np.divide(1,np.square(distance_0),out=dz0_dd, where=distance_0!=0)

        dz1_dd = np.ones(distance_1.shape)
        dz1_dd = np.divide(1,np.square(distance_1),out=dz1_dd, where=distance_1!=0) # 4x4
        print(dz1_dd.shape)
        
        
        dz0_dX = np.ones(self.batch_features.shape)  # 4x32
        #1-32
        for h in range(self.batch_features.shape[1]):
            dz_dx = np.ones(self.distance.shape)  # 4x4
            #1-4
            for i in range(self.distance.shape[0]):
                #1-4
                for j in range(self.distance.shape[1]):
                    x_diff = self.batch_features[i,h] - self.batch_features[j,h]  # 1x1
                    den = self.distance[i,j]  # 1x1
                    #print(x_diff,den)
                    dz_dx[i,j] = np.nan_to_num(x_diff/den)  # 1x1
                    #print(np.nan_to_num(dz_dx[i,j]))
            dz0_dX[:,h] = (dz0_dd * dz_dx).sum(axis=1)  # 4,   - this replaces a col in the mtx
            print(dz0_dX, dz0_dX[:,h])
            print('-------------------------------')
        print(dz0_dX.shape)

        
        dz1_dX = np.ones(self.batch_features.shape)
        for h in range(self.batch_features.shape[1]):
            dz_dx = np.ones(self.distance.shape)
            for i in range(self.distance.shape[0]): 
                for j in range(self.distance.shape[1]):
                    x_diff = self.batch_features[i,h] - self.batch_features[j,h]
                    den = self.distance[i,j]
                    #print(x_diff,den)
                    dz_dx[i,j] = np.nan_to_num(x_diff/den)
                    #print(np.nan_to_num(dz_dx[i,j]))
            dz1_dX[:,h] = (dz1_dd * dz_dx).sum(axis=1)
        
        ds1_dX = tf.math.multiply(np.array(ds1_dz0).reshape(4,1), dz0_dX) + tf.math.multiply(np.array(ds1_dz1).reshape(4,1), dz1_dX)
        dL_dX = tf.math.multiply(np.array(dL_dS).reshape(4,1), ds1_dX)
        #print("Feature Gradients:")
        #print(dL_dX)
        
        return dL_dX
    
    def pairwise_dist(self, A, B):  
        """
        Computes pairwise distances between each elements of A and each elements of B.
        Args:
        A,    [m,d] matrix
        B,    [n,d] matrix
        Returns:
        D,    [m,n] matrix of pairwise distances
        """
        with tf.compat.v1.variable_scope('pairwise_dist'):
            # squared norms of each row in A and B
            na = tf.reduce_sum(tf.square(A), 1)
            nb = tf.reduce_sum(tf.square(B), 1)

            # na as a row and nb as a co"lumn vectors
            na = tf.reshape(na, [-1, 1])
            nb = tf.reshape(nb, [1, -1])

            # return pairwise euclidead difference matrix
            D = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0.0))
        return D

In [153]:
arun = arun_knn()
pred_labels = arun.forward(test_output, test_labels)
print(pred_labels[:,0])

tf.Tensor([0.37628729 0.63295432 0.61879672 0.36631772], shape=(4,), dtype=float64)


In [154]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
TL = loss(test_labels, pred_labels[:,0])
TL

<tf.Tensor: shape=(), dtype=float64, numpy=0.7894553542137146>

In [155]:
arun.backward(TL)

(4, 4)
[[ 0.20615569  1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.        ]
 [ 0.65535461  1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.        ]
 [-0.21293823  1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          1.          1.          1.          1.          1.
   1.          

<tf.Tensor: shape=(4, 32), dtype=float64, numpy=
array([[ 0.07005278, -0.02396257, -0.03147325,  0.01598121, -0.00303616,
         0.11684204, -0.05181865,  0.02816463,  0.12601119, -0.03319223,
         0.09414004, -0.0065908 , -0.00023258,  0.04637194, -0.04285128,
        -0.01072858,  0.05827691,  0.14677306,  0.01538665,  0.04779649,
         0.08346587, -0.04053073, -0.01903454, -0.01979521,  0.00540248,
         0.00552906,  0.16048012, -0.12661559, -0.13493421, -0.02230032,
         0.10428466, -0.09339834],
       [-0.06492621,  0.02335158,  0.02949927, -0.01373201,  0.00083373,
        -0.1092197 ,  0.04940009, -0.02679657, -0.11922344,  0.03212619,
        -0.08983084,  0.00471903,  0.00130728, -0.04568156,  0.0425797 ,
         0.01140116, -0.05466196, -0.1389033 , -0.01437243, -0.04537216,
        -0.07909436,  0.03757788,  0.01742589,  0.01876884, -0.00582138,
        -0.00494642, -0.15198702,  0.12076753,  0.12755275,  0.02066083,
        -0.09923443,  0.08950527],
     